In [ ]:
#Note: the setup for this script does not work, so section of code have been used to generate a new scripts

In [2]:
# required modules
import numpy as np
import requests
import json
from pprint import pprint
import sys
import pandas as pd
import pandas.io.sql as sql
import MySQLdb as mdb

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# This utility curates the names in PFR Compound Data by changing names to match the names in the Compound using CAS
# 1) load all the compounds from the Compound table into a dataframe
# 2) process through each compound and update the names in all records in the PFR Compound Data using CAS as the criteria

In [51]:
#this cell loads the compounds table from cam_data into a dataframe
try:   
    db_con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
    compounds_table = pd.read_sql('SELECT * FROM compounds', con=db_con)
except mdb.Error, e:
    print "Error %d: %s" % (e.args[0],e.args[1])
    con.rollback() #Rollback in case there is an error
finally:
    if db_con:
        db_con.close() #disconnect from database
data_table = compounds_table[['compound_name', 'cas']]
data_table = data_table[data_table.cas != ''] #remove rows that do not contain a cas number
data_table = data_table.head(10) #slice first 10 rows
data_table

,compound_name,cas
0,chlorogenic acid,327-97-9
1,phloridzin,60-81-1
2,epicatechin,490-49-0
3,catechin,154-24-4
5,quercetin 3-rutinoside,153-18-4
6,quercetin 3-galactoside,482-36-0
7,quercetin 3-rhamnoside,522-12-3
8,procyanidin B2,29106-49-8
9,citric acid,77-92-9
11,apigenin,520-36-5


In [19]:
compounds_table[['compound_name', 'cas']]

,compound_name,cas
0,chlorogenic acid,327-97-9
1,phloridzin,60-81-1
2,epicatechin,490-49-0
3,catechin,154-24-4
4,-,
5,quercetin 3-rutinoside,153-18-4
6,quercetin 3-galactoside,482-36-0
7,quercetin 3-rhamnoside,522-12-3
8,procyanidin B2,29106-49-8
9,citric acid,77-92-9


In [13]:
#set up for generating the sql query with %s formating.
table = 'key_words'
keyword = 'epicatechin'
insert_record = "INSERT INTO %s (key_word) VALUES ('%s')" % (table, keyword)
#insert_record = "INSERT INTO key_words (key_word) VALUES ('%s')" % (keyword)
print insert_record

INSERT INTO key_words (key_word) VALUES ('epicatechin')


In [28]:
    try:   
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor(mdb.cursors.DictCursor)
        update_record = "UPDATE compounds SET canonical_smiles='C1C(C(C(CC1(C(=O)O)O)OC(=O)C=CC2=CC(=C(C=C2)O)O)O)O' WHERE pub_chem=1794427" 
        cur.execute(update_record) # Execute the SQL command
        con.commit() # Commit your changes in the database 
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database

In [36]:
#Database access functions

def UpdateCompoundNamebyCAS(table, name, cas_num):
    try:   
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor(mdb.cursors.DictCursor)
        update_record = "UPDATE %s SET assigned_name = '%s' WHERE cas = '%s'" % (table, name, cas_num)
        print update_record
        cur.execute(update_record) # Execute the SQL command
        con.commit() # Commit your changes in the database 
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def SelectTableToDF(table):
    try:   
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor(mdb.cursors.DictCursor)
        get_record = "SELECT * FROM %s" % (table)
        cur.execute(get_record) # Execute the SQL command
        record=cur.fetchone()
        print record['compound_name'], record['cas']
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def SelectOneRecord(table, value):
    try:   
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor(mdb.cursors.DictCursor)
        get_record = "SELECT * FROM %s WHERE pub_chem=%s" % (table, value)
        cur.execute(get_record) # Execute the SQL command
        record=cur.fetchone()
        print record['compound_name'], record['cas']
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def InsertOneCompound(value1, value2, value3, value4, value5, value6, value7, value8, value9): #insets a new record into the compounds 
    try:   
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor(mdb.cursors.DictCursor)
        insert_record = "INSERT INTO compounds (compound_name, cas, formula, exact_mass, pub_chem, chemspider_id, isomeric_smiles, comment, pseudonyms) VALUES ('%s', '%s','%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (value1, value2, value3, value4, value5, value6, value7, value8, value9)
        cur.execute(insert_record) # Execute the SQL command
        con.commit() # Commit your changes in the database 
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def UpdateOneRecord(table, c_smiles, i_smiles, pc_id):
    try:   
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor(mdb.cursors.DictCursor)
        update_record = "UPDATE %s SET canonical_smiles='%s', isomeric_smiles='%s' WHERE pub_chem=%s" % (table, c_smiles, i_smiles, pc_id)
        cur.execute(update_record) # Execute the SQL commac_smilesnd
        con.commit() # Commit your changes in the database 
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def WriteOneValueToTable(table, column, value):
    try:
        con = mdb.connect('localhost', 'root', 'abc123', 'excel_index');
        cur = con.cursor()
        insert_record = "INSERT INTO %s (%s) VALUES ('%s')" % (table, column, value)
        cur.execute(insert_record) # Execute the SQL command
        con.commit() # Commit your changes in the database    
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def WriteTwoValuesToTable(table, column1, column2, value1, value2):
    try:
        con = mdb.connect('localhost', 'root', 'abc123', 'excel_index');
        cur = con.cursor()
        insert_record = "INSERT INTO %s (%s, %s) VALUES ('%s', '%s')" % (table, column1, column2, value1, value2)
        cur.execute(insert_record) # Execute the SQL command
        con.commit() # Commit your changes in the database    
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def RecordExists(table, column, value):
    try:
        con = mdb.connect('localhost', 'root', 'abc123', 'cam_data');
        cur = con.cursor()
        find_record = "SELECT COUNT(1) FROM %s WHERE %s='%s'" % (table, column, value)
        cur.execute(find_record) # Execute the SQL command
        if cur.fetchone()[0]:
            return True
        else:
            return False
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return

def FindID(table, column, value):
    try:
        # from http://zetcode.com/db/mysqlpython/
        #cur = con.cursor(mdb.cursors.DictCursor)
        #cur.execute("SELECT * FROM Writers LIMIT 4")
        #rows = cur.fetchall()

        #for row in rows:
            #print row["Id"], row["Name"]
      
        con = mdb.connect('localhost', 'root', 'abc123', 'excel_index');
        cur = con.cursor(mdb.cursors.DictCursor)
        get_record = "SELECT * FROM %s WHERE %s='%s'" % (table, column, value)
        cur.execute(get_record) # Execute the SQL command
        rows=cur.fetchall()
        col_title="id_" + column
        print col_title, column
        for row in rows:
            print row[col_title], row[column] 
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        con.rollback() #Rollback in case there is an error
    finally:
        if con:
            con.close() #disconnect from database
    return


In [ ]:
table = 'compoundpfr_data'
new_name = 'chlorogenic acid'
cas_num = '327-97-9'
UpdateCompoundNamebyCAS(table, new_name, cas_num)

In [56]:
# iterate the rows of the dataframe and write the first value in each row to a database
table = 'compoundpfr_data'
column = 'cas'
for i,r in data_table.iterrows():
    cas_present = 'cas absent'
    new_name = data_table.ix[i,'compound_name'] #select the first column of each row
    cas_num = data_table.ix[i,'cas'] #select the first column of each row
    if RecordExists(table, column, cas_num):
        cas_present = 'CAS present'
        UpdateCompoundNamebyCAS(table, new_name, cas_num)
    print i, new_name, cas_num, cas_present
print 'Finished!'

UPDATE compoundpfr_data SET assigned_name = 'chlorogenic acid' WHERE cas = '327-97-9'
0 chlorogenic acid 327-97-9 CAS present
UPDATE compoundpfr_data SET assigned_name = 'phloridzin' WHERE cas = '60-81-1'
1 phloridzin 60-81-1 CAS present
UPDATE compoundpfr_data SET assigned_name = 'epicatechin' WHERE cas = '490-49-0'
2 epicatechin 490-49-0 CAS present
UPDATE compoundpfr_data SET assigned_name = 'catechin' WHERE cas = '154-24-4'
3 catechin 154-24-4 CAS present
UPDATE compoundpfr_data SET assigned_name = 'quercetin 3-rutinoside' WHERE cas = '153-18-4'
5 quercetin 3-rutinoside 153-18-4 CAS present
UPDATE compoundpfr_data SET assigned_name = 'quercetin 3-galactoside' WHERE cas = '482-36-0'
6 quercetin 3-galactoside 482-36-0 CAS present
UPDATE compoundpfr_data SET assigned_name = 'quercetin 3-rhamnoside' WHERE cas = '522-12-3'
7 quercetin 3-rhamnoside 522-12-3 CAS present
UPDATE compoundpfr_data SET assigned_name = 'procyanidin B2' WHERE cas = '29106-49-8'
8 procyanidin B2 29106-49-8 CAS pr

In [35]:
# this cell opens csv file containing a down load of urinary metabolites from HMDB
# The delimiter must be tab
# Each metabolite is read by row as a string and then split into components
# If the metabolite cas number does not exists in camdata>compounds the metabolite is inserted
infile = open("/home/tony/pythonNotebooks/data/HMDB_Urinary_Metabolites_tab.csv", "r")
db_table="compounds"
table_col="cas"
for line in infile.readlines():
    #c = line.split(",")		#split line by commas to get individual strings
    c = line.split("\t")		#split line by tabs to get individual strings
    hmdb_id=c[0]
    cmpd_name=c[1]
    synonyms=c[2]
    cas=c[3]
    isomeric_smiles=c[4]
    formula=c[5]
    mass=c[6]
    pubchem_id=c[7]
    cs_id=c[8]
    comment="HMDB urinary metabolite"
    print hmdb_id, cmpd_name, cas, formula, mass, pubchem_id, cs_id
    if RecordExists(db_table,table_col,cas):
        print 'compound %s in table' % (cmpd_name)
    else:
        print 'compound %s NOT in table: inserting into compounds' % (cmpd_name)
        InsertOneCompound(cmpd_name, cas, formula, mass, pubchem_id, cs_id, isomeric_smiles, comment, synonyms)
       
infile.close()


HMDB ID Name CAS Number Chemical Formula Monoisotopic Mass PubChem Compound ID ChemSpider ID

compound Name in table
HMDB00001 1-Methylhistidine 332-80-9 C7H11N3O2 169.085126611 92105 83153

compound 1-Methylhistidine in table
HMDB00002 1,3-Diaminopropane 109-76-2 C3H10N2 74.08439833 428 415

compound 1,3-Diaminopropane in table
HMDB00005 2-Ketobutyric acid 600-18-0 C4H6O3 102.031694058 58 57

compound 2-Ketobutyric acid in table
HMDB00008 2-Hydroxybutyric acid 600-15-7 C4H8O3 104.047344122 11266 10792

compound 2-Hydroxybutyric acid in table
HMDB00010 2-Methoxyestrone 362-08-3 C19H24O3 300.172544634 440624 389515

compound 2-Methoxyestrone in table
HMDB00011 (R)-3-Hydroxybutyric acid 625-72-9 C4H8O3 104.047344122 92135 83181

compound (R)-3-Hydroxybutyric acid in table
HMDB00012 Deoxyuridine 951-78-0 C9H12N2O5 228.074621504 13712 13118

compound Deoxyuridine in table
HMDB00014 Deoxycytidine 951-77-9 C9H13N3O4 227.090605919 13711 13117

compound Deoxycytidine in table
HMDB00015 Cortexo

-c:56: Warning: Field 'sys_name' doesn't have a default value
-c:56: Warning: Field 'metlin_id' doesn't have a default value
-c:56: Warning: Field 'compound_type' doesn't have a default value
-c:56: Warning: Field 'chemfinder_ref' doesn't have a default value
-c:56: Warning: Field 'canonical_smiles' doesn't have a default value


In [16]:
# 
# This is cell not part of the script, but left as an example of open in excel file and iterating through the rows of a dataframe
# 
# opens an excel file and load the first sheet as a dataframe
# extracts keywords from the dataframe; writes to keyword table if does not exist; if exists writes to found_key_words and scanned_files tables
#DataFileName='compounds.xlsx' # test spreadsheet with compound names
#df_xls=xls_file.parse('Sheet1')
DataFileName='MH11_UPLC_processedData_250.xlsx' #test spreadsheet from a real dataset that contain a 'Cover Sheet'
xls_file=pd.ExcelFile(DataFileName)
df_xls=xls_file.parse('Cover Sheet')
print DataFileName
WriteOneValueToTable('scanned_files', 'filename', DataFileName)

#df_xls
# iterate the rows of the dataframe and write the first value in each row to a database
for i,r in df_xls.iterrows():
    #print i, df_xls.ix[i,0]
    compound=df_xls.ix[i,1] #select the first column of each row
    compound=compound.split(' ')
    # the command above fails if an integer is read, because there is no  split attribute for an integer
    #compound=compound.replace(' ','') #remove spaces as they are not alpha or num
    #print compound
    for word in compound: # iterate though the list of words generated by the split and write to table if not already there
        print word
        if word.isalpha():
            if not RecordExists('key_words', 'key_word', word):
                WriteOneValueToTable('key_words', 'key_word', word)
                print word, "not found; added to key_words"
            else:
                print word, "found; not added to key_words"
            # this does not work need to input integers, that is ids.  Need to write a funtion to get ids
            #WriteTwoValuesToTable('found_key_words','id_keyword', 'id_scanned_files', word, DataFileName)
print 'Finished!'

IOError: [Errno 2] No such file or directory: 'MH11_UPLC_processedData_250.xlsx'